<a href="https://colab.research.google.com/github/ToshiroHJJZ/Detection-and-Classification-of-Linguistic-Features-for-Effective-Spam-Identification/blob/main/Adversial_attack_Ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

data = pd.read_csv("/content/drive/MyDrive/HThesis/adversarial_social_media_dataset.csv", encoding='latin-1')


In [ ]:
import random
import nltk
from nltk.corpus import wordnet

# Download wordnet for synonym replacement if not already downloaded
nltk.download('wordnet')


ham_df = pd.DataFrame(data)

def get_synonym(word):
    """Get a synonym for a word if available."""
    synonyms = wordnet.synsets(word)
    if synonyms:
        for lemma in synonyms[0].lemmas():
            synonym = lemma.name()
            if synonym.lower() != word.lower():
                return synonym
    return word

def synonym_replacement(text, num_replacements=1):
    """Replace words in the text with synonyms."""
    words = text.split()
    new_words = words[:]
    random_word_list = [word for word in words if wordnet.synsets(word)]
    random.shuffle(random_word_list)

    num_replaced = 0
    for random_word in random_word_list:
        synonym = get_synonym(random_word)
        if synonym != random_word:
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
            if num_replaced >= num_replacements:
                break
    return ' '.join(new_words)

def char_swap(text, num_swaps=1):
    """Randomly swap adjacent characters in the text."""
    text = list(text)
    for _ in range(num_swaps):
        idx = random.randint(0, len(text) - 2)
        text[idx], text[idx + 1] = text[idx + 1], text[idx]
    return ''.join(text)

def random_insertion(text, num_insertions=1):
    """Insert spammy words randomly into the text."""
    spammy_words = ["limited", "urgent", "winner", "click", "offer", "gift",
    "earn", "billion", "cash", "free",  "giveaway", "click link", "limited offer", "flash sale", "exclusive deal",
    "win big", "free gift", "act fast", "claim your prize", "urgent",
    "limited time", "last chance", "click now", "swipe up", "must see",
    "free entry", "special offer", "don’t miss", "win now", "unlock",
    "amazing deal", "get it here", "check this out", "viral",
    "guaranteed winner", "limited spots", "claim free", "join now", "only today",
    "don’t wait", "biggest sale", "double tap", "repost to win", "follow us",
    "secret offer", "instant access", "see details", "claim now", "gift card",
    "link in bio", "only for you", "just for you", "get yours", "cash prize",
    "redeem now", "early access", "exclusive content", "click to learn more",
    "personal message", "free shipping", "limited stock", "real winner",
    "too good to miss", "you’ve been selected", "view profile", "lucky winner"]
    words = text.split()
    for _ in range(num_insertions):
        spam_word = random.choice(spammy_words)
        insert_pos = random.randint(0, len(words))
        words.insert(insert_pos, spam_word)
    return ' '.join(words)

def apply_adversarial_attack(text):
    """Apply a combination of adversarial attacks to the text."""
    text = synonym_replacement(text, num_replacements=1)
    text = char_swap(text, num_swaps=1)
    text = random_insertion(text, num_insertions=1)
    return text

# Create a new DataFrame with attacked versions of the ham messages
attacked_data = []

for index, row in ham_df.iterrows():
    # Assuming the column containing the message is named 'message' instead of 'v2'
    original_message = row["text"]

    # Append the original message with non-attacked label
    # Changed from: attacked_data.append({"text": original_message, "text": 0})
    attacked_data.append({"text": original_message, "label": 0})  # Correctly assign label

    # Create an attacked version of the message and add to the data
    attacked_message = apply_adversarial_attack(original_message)

    # Append attacked message with attacked label
    attacked_data.append({"text": attacked_message, "label": 1})

# Convert to DataFrame
attacked_df = pd.DataFrame(attacked_data)

# Display the new DataFrame
print(attacked_df)

# optionally save the datset to csv
location = "/content/drive/MyDrive/HThesis/"

attacked_df.to_csv("ham_attacked_non_attacked.csv", index=False)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                    text  label
0      Go until jurong point, crazy.. Available only ...      0
1      Go until jurong point, crazy.. Available only ...      1
2                          Ok lar... Joking wif u oni...      0
3                  Ok lar... get yours joke iwf u oni...      1
4      Free entry in 2 a wkly comp to win FA Cup fina...      0
...                                                  ...    ...
22283  Pity, * Washington in mood billion for that. S...      1
22284  The guy did some bitching but I acted like i'd...      0
22285  The guy did some bitching ubt I act like i'd b...      1
22286                         Rofl. Its true to its name      0
22287  Rofl. information_technology treu to gift card...      1

[22288 rows x 2 columns]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset (assuming you've saved it as 'ham_attacked_non_attacked.csv')
data = pd.read_csv("/content/ham_attacked_non_attacked.csv")

# Step 1: Preprocess the text data with TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['text']).toarray()
y = data['label']

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Step 4: Make predictions and evaluate the model
y_pred = rf_classifier.predict(X_test)

# Print the evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 0.9697173620457604
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2241
           1       0.97      0.97      0.97      2217

    accuracy                           0.97      4458
   macro avg       0.97      0.97      0.97      4458
weighted avg       0.97      0.97      0.97      4458



Multiple models

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Define models
models = {
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Bagging Classifier': BaggingClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

#creates a list to store the results
results = []

# Train and evaluate each model
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)  # Get report as a dictionary
    `   e
display(results_df)

,Model,Accuracy,Precision,Recall,F1-Score
0,K-Nearest Neighbors,0.492598,0.396131,0.492598,0.343828
1,Logistic Regression,0.926649,0.927544,0.926649,0.926600
2,Decision Tree,0.974877,0.975368,0.974877,0.974868
3,Random Forest,0.970615,0.970619,0.970615,0.970614
4,Support Vector Machine,0.937416,0.938822,0.937416,0.937355
5,Bagging Classifier,0.973082,0.973421,0.973082,0.973075
6,Gradient Boosting,0.910274,0.914800,0.910274,0.910000
